Here we are going to look at **git as purely local file management**.  The next one looks at working with the server. 

**Before we start** remove any pre-existing version of the dummy project that we'll create and see what our git configuration looks like

In [1]:
#!pwsh
cd ~
del dummyProj -rec -force
git config --list

http.sslbackend=schannel
diff.astextplain.textconv=astextplain
core.autocrlf=true
core.fscache=true
core.symlinks=false
core.editor="C:\\Program Files\\Microsoft VS Code\\Code.exe" --wait
credential.helper=manager
filter.lfs.clean=git-lfs clean -- %f
filter.lfs.smudge=git-lfs smudge -- %f
filter.lfs.process=git-lfs filter-process
filter.lfs.required=true
pull.rebase=false
user.name=James O'Neill
user.email=P10506111@capita.co.uk
credential.helper=wincred

Lets **Initialize a new local repo**.    
_What's a "repo"_ - a Repository is a collection of files tracked by git. It's all the files and folders under a given path - that might be a local directory, or a URL to a server. 
Within the repo git knows about "blobs" which are versions of files, and it knows the blob that represents the active version of each file in the repo. Git knows which files existed in the repo at differnt points in time and which blobs represented them at that time (giving us the ability to move backwards and forwards in file history ) 

So let's create one:
- Make a directory, 
- move into it 
- run `git init`

In [1]:
#!pwsh
$null = md dummyProj 
cd ~\dummyProj  
git init

Initialized empty Git repository in C:/Users/P10506111/dummyProj/.git/

When we add a file, initially it is in the directory, but git hasn't made a blob for it. So let's
- Make a file,
- use `git add filename` to **add** it to tell Git to start copy versions of it into blobs
- make a **new commit** with a **message**  do the commit verbosly 

_what's a commit ?_   When I said before that Git knows which files existed in the repo at differnt points in time and which blobs represented them, a commit is saying "This is one of those points". So git records the contents of the directory, and which blob maps to which file.  

In [1]:
#!pwsh
cd ~\dummyProj 
echo "Placeholder" > license.md 
git add license.md  
git commit -v -m "Inital version"  

[master (root-commit) ef2364f] Inital version
 1 file changed, 1 insertion(+)
 create mode 100644 license.md

Git keeps a log so let's see what's in the **log** so far. 

In [1]:
#!pwsh
cd ~\dummyProj 
git log

commit ef2364fbe471899c2267afd08654bd53e0e0a8da
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Thu Sep 3 17:13:11 2020 +0100

    Inital version

We can get a shorter view with `--oneline`

In [1]:
#!pwsh
cd ~\dummyProj 
git log --oneline

ef2364f Inital version

_Side note from https://git-scm.com/book/en/v2/Git-Basics-Getting-a-Git-Repository_

To **clone** the Git linkable library called libgit2, you can do so like this:    
`git clone https://github.com/libgit2/libgit2`    
That creates a directory named `libgit2`, initializes a `.git` directory inside it, pulls down all the data for that repository, and checks out a working copy of the latest version.   
To clone the repository into a directory named something other than libgit2, you can specify the new directory name as an additional argument:    
`git clone https://github.com/libgit2/libgit2 mylibgit`    

Whether we used **clone** or **init** we now have a git enabled folder - in other words one with a .git directory which is where the blobs and commit information live . Let's check its **status**

In [1]:
#!pwsh
cd ~/dummyProj 
git status  

On branch master
nothing to commit, working tree clean

The status will change if we add, update or delete files, so let's extend an existing file and add a copy of it, and get the status again.

In [1]:
#!pwsh
cd ~/dummyProj 
echo 'more' >> license.md 
copy license.md readme
git status  

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   license.md

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	readme

no changes added to commit (use "git add" and/or "git commit -a")

Git tells us our changes are in the form of one additional  file, currently **untracked**, and one **modified** file which has not yet be staged for committing. `add` will update what is going to be committed, `restore` will roll back to a previous version and `commit -a` is like doing `add *` followed by `commit`

We can see the changes to existing files with `diff`

In [1]:
#!pwsh
git diff

diff --git a/license.md b/license.md
index 3b94f91..ce2ca9f 100644
--- a/license.md
+++ b/license.md
@@ -1 +1,2 @@
 Placeholder
+more

`git add *` will **stage** all new and changed files, and we can check the status again.

In [1]:
#!pwsh
git add *  
git status

On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   license.md
	new file:   readme


As before, `restore` can be used to to reverse things, `restore --staged` moves the file back (undoing any changes since `git add` was run) and removes it from the list of files to include in the next commit.

The `diff` command can show us what has changed - on it's own it shows what is *tracked*, *unstaged* files, and all our files are *staged* now so that won't show anything `--cached` shows what's staged and waiting to be committed. 

In [1]:
#!pwsh
git diff --cached

diff --git a/license.md b/license.md
index 3b94f91..ce2ca9f 100644
--- a/license.md
+++ b/license.md
@@ -1 +1,2 @@
 Placeholder
+more
diff --git a/readme b/readme
new file mode 100644
index 0000000..ce2ca9f
--- /dev/null
+++ b/readme
@@ -0,0 +1,2 @@
+Placeholder
+more

Let's `commit` the the change  with a new message (`-m`) and see what is in the log afterwards

In [1]:
#!pwsh
git commit -m "two" 

[master e696360] two
 2 files changed, 3 insertions(+)
 create mode 100644 readme

In [1]:
#!pwsh
git log --oneline

e696360 two
ef2364f Inital version

So the **log shows the history of every commit** which got us to this point. 
Let's remove a file now

In [1]:
#!pwsh
del .\license.md
echo 'extra' >> .\readme
git status

On branch master
Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    license.md
	modified:   readme

no changes added to commit (use "git add" and/or "git commit -a")

This time the status shows a deleted file. We might be temped to use the `git rm` command to remove all deleted files, but beware...

In [1]:
#!pwsh
git rm *

error: the following file has local modifications:
    readme
(use --cached to keep the file, or -f to force removal)

`rm *` _did_ try to remove everything. Fortunately our modified file saved us. Let's try just remove the file which needs deleting, and add the other changed files. Then let's try a new option for the status command. 

In [1]:
#!pwsh
git rm ./license.md
git add *

rm 'license.md'

In [1]:
#!pwsh
git status -s 

D  license.md
M  readme

to keep the file in your working tree but remove it from your staging area  e.g accidentally staged    
`git rm --cached README`   
https://git-scm.com/book/en/v2/Git-Basics-Recording-Changes-to-the-Repository 

Missed a file ? add it & run `git commit --amend`  

For now lets commit the deletion(s) and update(s) and then see what has changed in the log

In [1]:
#!pwsh
git commit -m "three"  

[master 32fc27d] three
 2 files changed, 3 insertions(+), 2 deletions(-)
 delete mode 100644 license.md

In [1]:
#!pwsh
git log

commit 32fc27d19fda3f7d6a853d553eff501f1c9da0d8
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Thu Sep 3 18:30:37 2020 +0100

    three

commit e696360a2bab11e28e006c22f436666bb47d7e31
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Thu Sep 3 17:20:55 2020 +0100

    two

commit ef2364fbe471899c2267afd08654bd53e0e0a8da
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Thu Sep 3 17:13:11 2020 +0100

    Inital version

We can move or rename files and follow the operation with    
`git rm <oldName>`    
`git add <NewName>` 
but `git mv` does all 3 in one 

In [1]:
#!pwsh
git mv .\readme read_me.txt  
git status -s

R  readme -> read_me.txt

git log -p -1

In [1]:
#!pwsh
git log --stat --since=1.hours  

commit 32fc27d19fda3f7d6a853d553eff501f1c9da0d8
Author: James O'Neill <P10506111@capita.co.uk>
Date:   Thu Sep 3 18:30:37 2020 +0100

    three

 license.md | 2 --
 readme     | 3 +++
 2 files changed, 3 insertions(+), 2 deletions(-)

# Branches
The line of commits forms a branch. In the early years of git, the first branch was always named "master". Other names are now allowed and people increasingly prefer other names like "main" or "trunk. We are currently working in that branch so  the **HEAD** pointer and current state of that branch map to the our most recent commit.  

In [1]:
#!pwsh
git log --oneline --decorate 

32fc27d (HEAD -> master) three
e696360 two
ef2364f Inital version

In this case the **HEAD** pointer and the current commit in the **master** branch point to the commit with the hash `32fc27d` which had the commit message **three** , this is descended from commit `e696360` ("two"), which descended from commit `ef2364f` ("Initial version") 

If we add a new branch it creates the name and with no other parameters starts pointing wherever **HEAD** points but **HEAD** does not switch to the new branch. 

In [1]:
#!pwsh
git branch testing
git log --oneline --decorate 

fatal: A branch named 'testing' already exists.
c24aeb3 (HEAD -> testing) changed the testing branch
32fc27d (master) three
e696360 two
ef2364f Inital version

 `git checkout <branchname>` selects a branch to work in - or we can create & select in one command with  `git checkout -b <branchname>`

In [1]:
#!pwsh
git checkout testing   

A	read_me.txt
Switched to branch 'testing'
D	readme

<u>Now</u> the **head** points to the new branch

In [1]:
#!pwsh
git log --oneline --decorate 

32fc27d (HEAD -> testing, master) three
e696360 two
ef2364f Inital version

If we add a file and commit our changes, the HEAD pointer and the current branch ("testing") move forard to the new commit. But any other branches (like "master") remain at their existing commits.

In [1]:
#!pwsh
dir -force > test.txt
git commit -a -m "changed the testing branch"

[testing c24aeb3] changed the testing branch
 1 file changed, 0 insertions(+), 0 deletions(-)
 rename readme => read_me.txt (100%)

In [1]:
#!pwsh
git log --oneline --decorate 

c24aeb3 (HEAD -> testing) changed the testing branch
32fc27d (master) three
e696360 two
ef2364f Inital version

We can switch back to the "master" branch, (changing where HEAD points), change its content and commit

In [1]:
#!pwsh
git checkout master

Switched to branch 'master'

In [1]:
#!pwsh
echo "new" > update.txt
git add *
git commit -m "changed master"

[master 4de7c34] changed master
 2 files changed, 11 insertions(+)
 create mode 100644 test.txt
 create mode 100644 update.txt

Another version of the `git log` command shows us both branches and how the overall stucture:

In [1]:
#!pwsh
git log --oneline --decorate --graph --all

* 4de7c34 (HEAD -> master) changed master
| * c24aeb3 (testing) changed the testing branch
|/  
* 32fc27d three
* e696360 two
* ef2364f Inital version

**HEAD** points to "master" and is currently at commit `4de7c34`; we have another branch "testing" which is at commit `c24aeb3` and both `4de7c34` and `c24aeb3` are dervived from `32fc27d` which in turn is derived from `e696360` and so on

Let's create a branch for a hot fix. 

In [1]:
#!pwsh
git checkout -b hotfix   
echo "hotfix" > hotfix.txt
git add *
git commit -m "added my hotfix"
git log --oneline --decorate --graph --all

Switched to a new branch 'hotfix'
[hotfix ac34b41] added my hotfix
 1 file changed, 1 insertion(+)
 create mode 100644 hotfix.txt
* ac34b41 (HEAD -> hotfix) added my hotfix
* 4de7c34 (master) changed master
| * c24aeb3 (testing) changed the testing branch
|/  
* 32fc27d three
* e696360 two
* ef2364f Inital version

We can build and deploy the hotfix using our new branch as the source of the code.     
Eventually we will want to merge those changes into the master branch.
Because there the  commit  for "hotfix" is a direct descentant of the commit for master, this is just a case of moving the name "master" along the line of descent - something git calls "fast forwarding"

In [1]:
#!pwsh
git checkout master   
git merge hotfix  

Switched to branch 'master'
Updating 4de7c34..ac34b41
Fast-forward
 hotfix.txt | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 hotfix.txt

In [1]:
#!pwsh
git log --oneline --decorate --graph --all

* ac34b41 (HEAD -> master, hotfix) added my hotfix
* 4de7c34 changed master
| * c24aeb3 (testing) changed the testing branch
|/  
* 32fc27d three
* e696360 two
* ef2364f Inital version

Now **HEAD** is attached to "master" which has moved forward from `4de7c34` to `ac34b41`; "hotfix" is still on `ac34b41` as well but it is a redundant label so we can delete it

In [1]:
#!pwsh
git branch -d hotfix 

Deleted branch hotfix (was ac34b41).

If we look at testing and master, we can see they have a common ancestor `32fc27d`. Testing has one extra commit which master doesn't have and lacks two commits which have been made to master - it is said to be "one ahead and two behind" . We can't do a fast forward merge but we can merge the changes from both  to make a single new commit with all the changes. 

In [1]:
#!pwsh
git checkout testing 
git merge master  

Switched to branch 'testing'
Merge made by the 'recursive' strategy.
 hotfix.txt |  1 +
 test.txt   | 10 ++++++++++
 update.txt |  1 +
 3 files changed, 12 insertions(+)
 create mode 100644 hotfix.txt
 create mode 100644 test.txt
 create mode 100644 update.txt

In this case the two branches have made changes which are independent of each other. Let's create a conflict. First let's update a file in the current branch ("Testing")

In [1]:
#!pwsh
type .\update.txt
echo "Added from the testing branch" >> update.txt
git commit -a -m 'Update from testing'

new
[testing e3c0b7e] Update from testing
 1 file changed, 1 insertion(+)

In [1]:
#!pwsh
git checkout master
type .\update.txt
''
echo "Added from the master branch" >> update.txt
git commit -a -m 'Update from master'
''
git log --oneline --decorate --graph --all

Switched to branch 'master'
new

[master f6517dd] Update from master
 1 file changed, 1 insertion(+)

* f6517dd (HEAD -> master) Update from master
| * e3c0b7e (testing) Update from testing
|/  
* ac34b41 added my hotfix
* 4de7c34 changed master
* 32fc27d three
* e696360 two
* ef2364f Inital version

As before the two branches have a common ancestor so let's try to merge them 

In [1]:
#!pwsh
git merge testing

Auto-merging update.txt
CONFLICT (content): Merge conflict in update.txt
Automatic merge failed; fix conflicts and then commit the result.

Oh dear - we need to manually edit update.txt which looks like this

In [1]:
#!pwsh
Type update.txt

new
<<<<<<< HEAD
Added from the master branch
Added from the testing branch
>>>>>>> testing

We can edit the file and either run `git merge --continue` (which will ask for the commit message to be edited) or use `git commit -a -m "merge message"`  and the log shows a new commit which brings together the shared ancestor and the updates.

In [1]:
#!pwsh
git log --oneline --decorate --graph --all

*   28238a1 (HEAD -> master) Merge branch 'testing'
|\  
| * e3c0b7e (testing) Update from testing
* | f6517dd Update from master
|/  
* ac34b41 added my hotfix
* 4de7c34 changed master
* 32fc27d three
* e696360 two
* ef2364f Inital version